# Logistic Regression to Classify Terrain by IMU and Odometry Data from TurtleBot3
### By Jacob Laframboise, Jack Demeter
Logistic regression works great when the data is randomly split into train and test (high 90 accuracy), but it struggles when the data is split into train/test based on which trial number is was collected in. This further supports the hypothesis that data from each run is more similar to itself than data from a given terrain. 



In [27]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.feature_selection import SelectKBest, chi2

import plotly as ply
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%matplotlib notebook

pd.set_option('display.max_columns', 120)
pd.set_option('display.max_rows', 80)

## Data Pre-Processing
We load in individual csv files collected from Rosbag on TurtleBot3 with ROS.

In [28]:
# Set dataFolder path to data file location
dataFolder = r"C:\Users\jaker\Documents\Experiment3Data-2019-11-21"

dataFiles = [
    r"gMitTile_s15_t8.csv",
    r"gMitTile_s15_t9.csv",
    r"gMitTile_s15_t10.csv",
    r"gTurf_s15_t3.csv",
    r"gTurf_s15_t4.csv",
    r"gTurf_s15_t5.csv",
    r"gTurf_s15_t6.csv",
    r"gTurf_s15_t7.csv",
    r"gTurf_s15_t8.csv",
    r"gTurf_s15_t9.csv",
    r"gTurf_s15_t10.csv",
    r"gArcTile_s15_t3.csv",
    r"gArcTile_s15_t4.csv",
    r"gArcTile_s15_t5.csv",
    r"gArcTile_s15_t6.csv",
    r"gArcTile_s15_t7.csv",
    r"gArcTile_s15_t8.csv",
    r"gArcTile_s15_t9.csv",
    r"gArcTile_s15_t10.csv",
    r"gCarp_s15_t3.csv",
    r"gCarp_s15_t4.csv",
    r"gCarp_s15_t5.csv",
    r"gCarp_s15_t6.csv",
    r"gCarp_s15_t7.csv",
    r"gCarp_s15_t8.csv",
    r"gCarp_s15_t9.csv",
    r"gCarp_s15_t10.csv",
    r"gMitTile_s15_t3.csv",
    r"gMitTile_s15_t4.csv",
    r"gMitTile_s15_t5.csv",
    r"gMitTile_s15_t6.csv",
    r"gMitTile_s15_t7.csv"
]

savePath = "Data-32Series-Delta30-Squared.csv"

In [29]:
""" 
For each data file we:
adjust the index, 
interpolate NaN values,
drop remaining NaN values, 
drop some empty columns.

We then augment the feature space with delta columns, 
and with polynomial columns,
and label the columns with terrain, speed, and trial number
"""
for i in range(len(dataFiles)):
    # get speed/terrain based on file name
    terrain = dataFiles[i].split('_')[0][1:]
    speed = dataFiles[i].split('_s')[1][:2]
    trial = dataFiles[i].split('_t')[1][0]

    df = pd.read_csv(os.path.join(dataFolder, dataFiles[i]))
    df = df.rename(columns={'Unnamed: 0': 'Seq'})
    df = df.set_index('Seq')

    # interpolate the missing data with a polynomial (upscaling)
    df = df.interpolate(method='polynomial', order=1)

    # remove incomplete entries
    df = df.dropna()

    # reset data to be ordered based on Sequence
    df = df.reset_index().drop(columns=['Seq'])

    
    df = df.drop(columns=['OdomPosZ', 'OdomOrientX', 'OdomOrientY', 'OdomLinY', 'OdomLinZ', 'OdomAngX', 'OdomAngY'])
    # use the XY magnitude to remove unique run IDs
    df['OdomPosXY'] = np.sqrt(df.OdomPosX**2 + df.OdomPosY**2)
    df = df.drop(columns=['OdomPosY', 'OdomPosX'])
    
    # Order is the exponent applied to the delta data sets to allow LR to find higher polynomial patterns, increases memory and comp. time
    order=2
    # the delta list specifies how many data points to go back to and apply the 
    dList = range(1, 302, 20)
    for col in df.columns.tolist():
        if col!='Sensor':
            for d in dList:
                df[col+'Delta{}'.format(d)] = df[col].diff(d)
                if order>1:
                    for p in range(2, order+1):
                        df[col+'Delta{}Exp{}'.format(d, p)] = df[col+'Delta{}'.format(d)]**p
        else:
            pass

    # remove the first ~302 entries with NaN in their delta columns
    df = df.dropna()
    df = df.reset_index().drop(columns=['index'])
    df = df.drop(columns=['Sensor', 'Time'])
    df['Speed']=int(speed)
    df['Terrain']=terrain
    df['Trial']=int(trial)

    if i==0:
        mainDf = df.copy(deep=True)
    else:
        mainDf = pd.concat([mainDf, df], axis=0, sort=False)
print("Data series completed.")
print("MainDf is now size {}".format(mainDf.shape))


Data series completed.
MainDf is now size (145353, 530)


In [30]:
# avoid reloading dataset when working, removing this can reduce memory usage (doubles)
df = mainDf.copy(deep=True)

In [31]:
# sample output of the data
df.head()

,OdomOrientZ,OdomOrientW,OdomLinX,OdomAngZ,ImuOrientX,ImuOrientY,ImuOrientZ,ImuOrientW,ImuAngVelX,ImuAngVelY,ImuAngVelZ,ImuAccelX,ImuAccelY,ImuAccelZ,OdomPosXY,TimeDelta1,TimeDelta1Exp2,TimeDelta21,TimeDelta21Exp2,TimeDelta41,TimeDelta41Exp2,TimeDelta61,TimeDelta61Exp2,TimeDelta81,TimeDelta81Exp2,TimeDelta101,TimeDelta101Exp2,TimeDelta121,TimeDelta121Exp2,TimeDelta141,TimeDelta141Exp2,TimeDelta161,TimeDelta161Exp2,TimeDelta181,TimeDelta181Exp2,TimeDelta201,TimeDelta201Exp2,TimeDelta221,TimeDelta221Exp2,TimeDelta241,TimeDelta241Exp2,TimeDelta261,TimeDelta261Exp2,TimeDelta281,TimeDelta281Exp2,TimeDelta301,TimeDelta301Exp2,OdomOrientZDelta1,OdomOrientZDelta1Exp2,OdomOrientZDelta21,OdomOrientZDelta21Exp2,OdomOrientZDelta41,OdomOrientZDelta41Exp2,OdomOrientZDelta61,OdomOrientZDelta61Exp2,OdomOrientZDelta81,OdomOrientZDelta81Exp2,OdomOrientZDelta101,OdomOrientZDelta101Exp2,OdomOrientZDelta121,...,ImuAccelZDelta61Exp2,ImuAccelZDelta81,ImuAccelZDelta81Exp2,ImuAccelZDelta101,ImuAccelZDelta101Exp2,ImuAccelZDelta121,ImuAccelZDelta121Exp2,ImuAccelZDelta141,ImuAccelZDelta141Exp2,ImuAccelZDelta161,ImuAccelZDelta161Exp2,ImuAccelZDelta181,ImuAccelZDelta181Exp2,ImuAccelZDelta201,ImuAccelZDelta201Exp2,ImuAccelZDelta221,ImuAccelZDelta221Exp2,ImuAccelZDelta241,ImuAccelZDelta241Exp2,ImuAccelZDelta261,ImuAccelZDelta261Exp2,ImuAccelZDelta281,ImuAccelZDelta281Exp2,ImuAccelZDelta301,ImuAccelZDelta301Exp2,OdomPosXYDelta1,OdomPosXYDelta1Exp2,OdomPosXYDelta21,OdomPosXYDelta21Exp2,OdomPosXYDelta41,OdomPosXYDelta41Exp2,OdomPosXYDelta61,OdomPosXYDelta61Exp2,OdomPosXYDelta81,OdomPosXYDelta81Exp2,OdomPosXYDelta101,OdomPosXYDelta101Exp2,OdomPosXYDelta121,OdomPosXYDelta121Exp2,OdomPosXYDelta141,OdomPosXYDelta141Exp2,OdomPosXYDelta161,OdomPosXYDelta161Exp2,OdomPosXYDelta181,OdomPosXYDelta181Exp2,OdomPosXYDelta201,OdomPosXYDelta201Exp2,OdomPosXYDelta221,OdomPosXYDelta221Exp2,OdomPosXYDelta241,OdomPosXYDelta241Exp2,OdomPosXYDelta261,OdomPosXYDelta261Exp2,OdomPosXYDelta281,OdomPosXYDelta281Exp2,OdomPosXYDelta301,OdomPosXYDelta301Exp2,Speed,Terrain,Trial
0,0.095387,0.995440,0.148886,-0.006150,-0.004592,-0.004974,-0.104227,-0.994526,0.000000,-0.027669,-0.013835,-0.290297,-0.049081,9.154829,18.806006,0.005092,0.000026,0.130681,0.017078,0.251098,0.063050,0.368369,0.135695,0.493427,0.243470,0.636363,0.404958,0.771192,0.594737,0.892386,0.796352,1.017193,1.034682,1.138310,1.295749,1.258815,1.584615,1.381064,1.907338,1.498555,2.245666,1.625103,2.640961,1.736605,3.015797,1.860827,3.462679,-0.000021,4.593532e-10,-0.000153,2.334641e-08,-0.000206,4.228962e-08,0.000041,1.700049e-09,0.000260,6.770568e-08,0.000506,2.555428e-07,0.000994,...,0.319260,-1.777096,3.158069,0.223260,0.049845,-2.354098,5.541779,-3.133411,9.818267,-1.962646,3.851981,-1.224035,1.498262,-1.312022,1.721402,-0.547673,0.299946,-0.365115,0.133309,0.575208,0.330864,-1.317110,1.734778,-1.399111,1.957512,-0.000008,6.955323e-11,-0.000277,7.655496e-08,-0.000521,2.718834e-07,-0.000789,6.224590e-07,-0.001140,0.000001,-0.001534,0.000002,-0.001878,0.000004,-0.002290,0.000005,-0.002780,0.000008,-0.003253,0.000011,-0.003737,0.000014,-0.004253,0.000018,-0.004768,0.000023,-0.005302,0.000028,-0.005779,0.000033,-0.006436,0.000041,15,MitTile,8
1,0.095366,0.995442,0.148886,-0.007478,-0.003933,-0.004679,-0.104174,-0.994536,0.008514,-0.064916,-0.013835,-0.099958,-0.174777,10.842142,18.805998,0.005561,0.000031,0.130839,0.017119,0.251223,0.063113,0.368386,0.135708,0.493495,0.243537,0.634416,0.402483,0.770069,0.593006,0.892173,0.795973,1.017388,1.035079,1.137589,1.294109,1.259750,1.586971,1.380945,1.907008,1.498703,2.246110,1.621436,2.629054,1.735753,3.012839,1.860233,3.460467,-0.000021,4.593532e-10,-0.000181,3.267875e-08,-0.000227,5.162486e-08,-0.000018,3.331765e-10,0.000254,6.460856e-08,0.000447,1.998104e-07,0.000953,...,0.060224,-0.089783,0.008061,1.910573,3.650289,-0.326809,0.106804,0.354342,0.125558,1.449689,2.101598,0.463278,0.214626,0.762553,0.581487,1.201290,1.443098,1.322198,1.748207,1.616685,2.613670,1.389235,1.929975,1.73

In [32]:
# check for none values and correct size
print(df.isnull().sum().sum()) # check for NaN
df.shape

0


(145353, 530)

### Feature engineering: elimination

In [33]:
# eliminate features that may ID individual runs improperly
# only retain delta data, speed, and terrain(the label)
columnsToDrop = [x for x in df.columns.tolist() if 'Time' in x or ('Delta' not in x and 'Speed' not in x and 'Terrain' not in x and 'Trial' not in x) ]
df = df.drop(columns=columnsToDrop)

In [34]:
# Group carp/turf and Tiles into 2 broader categories to attempt differentiation of similar terrains
df['Terrain'] = df['Terrain'].replace({'Carp':'Turf', 'ArcTile':'MitTile'})

### Speed filtering

In [35]:
df = df[df['Speed']==15]

### Logistic Regression
Now that we have processed the data, we can apply logistic regression to classify.

In [36]:
# sort the train test split where train on t1 test on t2
dfTrain = df[df['Trial']<9]
dfTest = df[df['Trial']>=9]

Y_train = dfTrain['Terrain']
Y_test = dfTest['Terrain']

X_train = dfTrain.drop(columns=['Terrain', 'Speed', 'Trial'])
X_test = dfTest.drop(columns=['Terrain', 'Speed', 'Trial'])

### Feature engineering: Kbest features

In [37]:
# highlight and utilize only the Kbest features in an attempt to reduce computation speed
featureCount = 30
test = SelectKBest(k=featureCount)
fit = test.fit(X_train, Y_train)

print(X_test.columns.tolist()[fit.scores_.argmax()])

# fit testing data
X_train = fit.transform(X_train)
X_test = fit.transform(X_test)

ImuAngVelXDelta21Exp2


In [38]:
# check proper sizing
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(123978, 30)
(123978,)
(21375, 30)
(21375,)


## Model running and accuracy exploration
During our testing an accuracy of ~75% was found when the data was split into two catergories, given that random guesses would result in ~50% accuracy the model did succeed, however not enough for the team to explore the additional applications that were initially intended for the data. Exploration with NN and other methods provided similar results.

In [39]:
model = LogisticRegression(solver = "lbfgs")
model.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
print("Accuracy on test set is: {}".format(model.score(X_test, Y_test)))

Accuracy on test set is: 0.7470877192982456
